# FaissStore

```{=mdx}

:::tip 兼容性
仅适用于 Node.js。
:::

```

[Faiss](https://github.com/facebookresearch/faiss) 是一个用于高效相似性搜索和密集向量聚类的库。

LangChain.js 支持使用 Faiss 作为本地运行的向量存储，且可以保存到文件中。它还提供了从 [LangChain Python 实现](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading) 读取已保存文件的能力。

本指南提供了关于如何开始使用 Faiss [向量存储](/docs/concepts/#vectorstores) 的快速概述。如需了解所有 `FaissStore` 功能和配置的详细文档，请查阅 [API 参考](https://api.js.langchain.com/classes/langchain_community_vectorstores_faiss.FaissStore.html)。

## 概述

### 集成详情

| 类 | 包 | [Python 支持](https://python.langchain.com/docs/integrations/vectorstores/faiss) | 包的最新版本 |
| :--- | :--- | :---: | :---: |
| [`FaissStore`](https://api.js.langchain.com/classes/langchain_community_vectorstores_faiss.FaissStore.html) | [`@langchain/community`](https://npmjs.com/@langchain/community) | ✅ |  ![NPM - 版本](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## 安装

要使用 Faiss 向量存储，你需要安装 `@langchain/community` 集成包以及将 [`faiss-node`](https://github.com/ewfian/faiss-node) 包作为对等依赖安装。

本指南还将使用 [OpenAI 嵌入](/docs/integrations/text_embedding/openai)，这要求你安装 `@langchain/openai` 集成包。如果需要，你也可以使用 [其他支持的嵌入模型](/docs/integrations/text_embedding)。

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community faiss-node @langchain/openai @langchain/core
</Npm2Yarn>
```

### 凭据

由于 Faiss 在本地运行，因此你无需任何凭据即可使用它。

如果你在本指南中使用 OpenAI 嵌入，则还需要设置你的 OpenAI 密钥：

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

如果你想对模型调用进行自动化追踪，也可以通过取消注释以下内容来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```typescript
// process.env.LANGSMITH_TRACING="true"
// process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

In [2]:
import { FaissStore } from "@langchain/community/vectorstores/faiss";
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const vectorStore = new FaissStore(embeddings, {});

## 管理向量存储

### 向向量存储中添加项目

In [3]:
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { source: "https://example.com" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { source: "https://example.com" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { source: "https://example.com" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { source: "https://example.com" }
}

const documents = [document1, document2, document3, document4];

await vectorStore.addDocuments(documents, { ids: ["1", "2", "3", "4"] });

[ '1', '2', '3', '4' ]


### 从向量存储中删除项目

In [4]:
await vectorStore.delete({ ids: ["4"] });

## 查询向量存储

一旦创建了向量存储并添加了相关文档，您很可能希望在链或代理运行期间查询它。

### 直接查询

执行一个简单的相似性搜索可以按如下方式进行：

In [5]:
const similaritySearchResults = await vectorStore.similaritySearch("biology", 2);

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* Mitochondria are made out of lipids [{"source":"https://example.com"}]


目前不支持按元数据筛选。

如果您要执行相似性搜索并获取相应的分数，可以运行：

In [6]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2);

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

* [SIM=1.671] The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* [SIM=1.705] Mitochondria are made out of lipids [{"source":"https://example.com"}]


### 通过转换为检索器进行查询

您还可以将向量存储转换为[检索器](/docs/concepts/retrievers)，以便在您的链中更方便地使用。

In [7]:
const retriever = vectorStore.asRetriever({
  k: 2,
});
await retriever.invoke("biology");

[
  {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { source: 'https://example.com' }
  },
  {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { source: 'https://example.com' }
  }
]


### 检索增强生成的用法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程：使用外部知识](/docs/tutorials/#working-with-external-knowledge)。
- [操作指南：使用 RAG 进行问答](/docs/how_to/#qa-with-rag)
- [检索概念文档](/docs/concepts/retrieval)

## 合并索引

Faiss 还支持合并现有索引：

In [ ]:
// Create an initial vector store
const initialStore = await FaissStore.fromTexts(
  ["Hello world", "Bye bye", "hello nice world"],
  [{ id: 2 }, { id: 1 }, { id: 3 }],
  new OpenAIEmbeddings()
);

// Create another vector store from texts
const newStore = await FaissStore.fromTexts(
  ["Some text"],
  [{ id: 1 }],
  new OpenAIEmbeddings()
);

// merge the first vector store into vectorStore2
await newStore.mergeFrom(initialStore);

// You can also create a new vector store from another FaissStore index
const newStore2 = await FaissStore.fromIndex(
  newStore,
  new OpenAIEmbeddings()
);

await newStore2.similaritySearch("Bye bye", 1);

## 将索引保存到文件并重新加载它

要将索引持久化到磁盘，请使用`.save`方法和静态的`.load`方法：

In [ ]:
// Create a vector store through any method, here from texts as an example
const persistentStore = await FaissStore.fromTexts(
  ["Hello world", "Bye bye", "hello nice world"],
  [{ id: 2 }, { id: 1 }, { id: 3 }],
  new OpenAIEmbeddings()
);

// Save the vector store to a directory
const directory = "your/directory/here";

await persistentStore.save(directory);

// Load the vector store from the same directory
const loadedVectorStore = await FaissStore.load(
  directory,
  new OpenAIEmbeddings()
);

// vectorStore and loadedVectorStore are identical
const result = await loadedVectorStore.similaritySearch("hello world", 1);
console.log(result);

## 从Python读取保存的文件

为了启用从[LangChain Python实现](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading)读取保存文件的功能，你需要安装[`pickleparser`](https://github.com/ewfian/pickleparser)包。

```{=mdx}
<Npm2Yarn>
  pickleparser
</Npm2Yarn>
```

然后你可以使用`.loadFromPython`静态方法：

In [ ]:
// The directory of data saved from Python
const directoryWithSavedPythonStore = "your/directory/here";

// Load the vector store from the directory
const pythonLoadedStore = await FaissStore.loadFromPython(
  directoryWithSavedPythonStore,
  new OpenAIEmbeddings()
);

// Search for the most similar document
await pythonLoadedStore.similaritySearch("test", 2);

## API 参考文档

如需详细了解所有 `FaissStore` 功能和配置，请访问 [API 参考文档](https://api.js.langchain.com/classes/langchain_community_vectorstores_faiss.FaissStore.html)